In [1]:
import os

# Check if we're in the root directory of rl-llm repo
if not os.path.basename(os.getcwd()) == 'rl-llm':
    # If we're in a subdirectory of rl-llm, find the root and cd to it
    current_path = os.getcwd()
    while os.path.basename(current_path) != 'rl-llm' and os.path.dirname(current_path) != current_path:
        current_path = os.path.dirname(current_path)
    
    if os.path.basename(current_path) == 'rl-llm':
        os.chdir(current_path)
        print(f"Changed directory to: {current_path}")
    else:
        print("Not in rl-llm repository structure")
else:
    print("Already in rl-llm root directory")

print(f"Current directory: {os.getcwd()}")

Changed directory to: /home/pavan/rl-llm
Current directory: /home/pavan/rl-llm


In [2]:
from src import EnvManager, sample_episodes
import time
import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig, get_peft_model
from trl import AutoModelForCausalLMWithValueHead

/home/pavan/rl-llm/.conda/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-GoTo-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/pavan/rl-llm/.conda/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Open-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/pavan/rl-llm/.conda/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Pickup-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/pavan/rl-llm/.conda/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-PutNext-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [3]:
model_id = "pavanpreet-gandhi/babyai-classical-ppo-prefinal-experiments-2025-04-11_13-38-03"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [5]:
import os
import torch
from transformers import AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead
from peft import PeftConfig

def load_model_with_peft_and_vhead(model_id, device="cuda"):
    """
    Load a model with both PEFT adapters and value head
    
    Args:
        model_id: Path to model or HF hub model ID
        device: Device to load the model to
    
    Returns:
        model: The loaded model with adapters and value head
        tokenizer: The associated tokenizer
    """
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Get PEFT config to find base model
    peft_config = PeftConfig.from_pretrained(model_id)
    base_model_id = peft_config.base_model_name_or_path
    
    print(f"Base model: {base_model_id}")
    print(f"Loading with PEFT adapters from: {model_id}")
    
    # Initialize model with value head from the base model
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        base_model_id,
        device_map="auto",
    )
    
    # Now load the PEFT adapter weights
    model = model.from_pretrained(model_id, device_map="auto")
    
    return model, tokenizer

# Example usage
model_id = "pavanpreet-gandhi/babyai-classical-ppo-prefinal-experiments-2025-04-11_13-38-03"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = load_model_with_peft_and_vhead(model_id, device)

# Test the model with a simple query
prompt = "You are in a room with a key. The instruction is to: pick up the key"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.7)
print("\nModel output:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# You can also access the value head 
with torch.no_grad():
    values = model.compute_value_estimates(
        inputs,
        max_new_tokens=50,
        return_dict=True
    )
    print(f"\nValue estimate: {values['values'].item():.4f}")

Base model: meta-llama/Llama-3.2-3B-Instruct
Loading with PEFT adapters from: pavanpreet-gandhi/babyai-classical-ppo-prefinal-experiments-2025-04-11_13-38-03


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


ValueError: The model is offloaded on CPU or disk - CPU & disk offloading is not supported for ValueHead models.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generation_kwargs = {
    "max_new_tokens": 20,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.8,
    "pad_token_id": tokenizer.pad_token_id,
}

env_id = "BabyAI-GoTo-v0"
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
    )
    for i in range(num_envs)
]
reasoning_flag = True

stats = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=1,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

NameError: name 'model' is not defined

## Base model (no finetuning)

Summary: BabyAI-GoTo-v0 with No Reasoning
Sample size: 20
Success Rate: 0.65
Average Reward: 0.51 ± 0.40
Average Episode Length: 23.10 ± 17.94
Average Number of Invalid Actions: 2.15

Summary: BabyAI-GoTo-v0 with Reasoning
Sample size: 20
Success Rate: 0.95
Average Reward: 0.77 ± 0.22
Average Episode Length: 14.80 ± 8.84
Average Number of Invalid Actions: 1.60

Summary: BabyAI-Pickup-v0 with No Reasoning
Sample size: 20
Success Rate: 0.55
Average Reward: 0.43 ± 0.41
Average Episode Length: 28.65 ± 21.20
Average Number of Invalid Actions: 2.30

Summary: BabyAI-Pickup-v0 with Reasoning
Sample size: 20
Success Rate: 0.70
Average Reward: 0.47 ± 0.36
Average Episode Length: 21.30 ± 15.40
Average Number of Invalid Actions: 2.50

Summary: BabyAI-PutNext-v0 with No Reasoning
Sample size: 20
Success Rate: 0.05
Average Reward: 0.01 ± 0.04
Average Episode Length: 78.90 ± 48.60
Average Number of Invalid Actions: 3.65

Summary: BabyAI-PutNext-v0 with Reasoning
Sample size: 21
Success Rate: 0.19
Average Reward: 0.07 ± 0.17
Average Episode Length: 95.81 ± 50.48
Average Number of Invalid Actions: 2.57

Summary: BabyAI-Open-v0 with No Reasoning
Sample size: 20
Success Rate: 0.35
Average Reward: 0.16 ± 0.25
Average Episode Length: 79.35 ± 34.19
Average Number of Invalid Actions: 3.50

Summary: BabyAI-Open-v0 with Reasoning
Sample size: 20
Success Rate: 0.30
Average Reward: 0.15 ± 0.25
Average Episode Length: 81.65 ± 48.86
Average Number of Invalid Actions: 3.05

Summary: BabyAI-PickUpSeqGoTo with No Reasoning
Sample size: 20
Success Rate: 0.55
Average Reward: 0.32 ± 0.32
Average Episode Length: 83.40 ± 39.94
Average Number of Invalid Actions: 0.90

Summary: BabyAI-PickUpSeqGoTo with Reasoning
Sample size: 20
Success Rate: 0.35
Average Reward: 0.19 ± 0.32
Average Episode Length: 84.50 ± 49.86
Average Number of Invalid Actions: 2.90


## Finetuned model (allowed to reason)

